In [1]:
import pandas as pd
import numpy as np
import neuroglia as ng


In [2]:
events = pd.read_csv('events.csv',index_col=0)

In [3]:
events.head()

,image_id,start,end,time
0,61,16095,16102,541.40872
1,115,16102,16109,541.64150
2,115,16110,16117,541.90753
3,40,16117,16124,542.14031
4,101,16125,16132,542.40635


In [4]:
timestamps = np.load('timestamps.npy')
dff = np.load('dff.npy')

In [5]:
traces = pd.DataFrame(
    dff.T,
    columns=range(dff.shape[0]),
    index=timestamps,
)
traces.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
6.18305,-0.091435,-0.057228,-0.113410,-0.018366,-0.058567,-0.017618,0.011481,0.056958,0.025251,0.071201,...,-0.075386,-0.064875,-0.071565,0.019540,0.064170,-0.144472,-0.130356,-0.075275,-0.056647,-0.030831
6.21630,-0.036901,-0.041414,-0.126635,-0.082418,-0.028359,-0.007267,0.002385,0.039544,0.060717,-0.003684,...,-0.048785,0.060133,-0.042139,0.019467,-0.037699,-0.077349,-0.002923,-0.065585,-0.042534,-0.066469
6.24956,-0.094552,-0.050752,-0.157663,-0.056625,0.032097,-0.082460,-0.022219,-0.047523,-0.011517,-0.050558,...,-0.038371,0.062461,0.054998,-0.009844,0.001376,-0.103788,0.126491,-0.060744,-0.080108,-0.136084
6.28281,-0.050839,-0.072834,-0.078211,-0.052841,-0.080964,-0.037020,0.003486,-0.017443,0.009277,-0.064238,...,-0.052811,0.020675,0.042299,-0.069828,0.082112,-0.049281,0.000692,-0.107399,0.005489,-0.031315
6.31606,-0.013625,-0.018929,-0.139947,-0.111368,0.004344,-0.057402,-0.082169,-0.012927,-0.046665,-0.053307,...,-0.050439,-0.056692,0.031534,-0.099023,-0.054193,-0.055991,0.023032,-0.032076,-0.017265,-0.016387


In [6]:
from neuroglia.event import MeanResponseExtractor

In [7]:
mean_response_extractor = MeanResponseExtractor(
    traces,
    bins=30,
    range=(0,1.0),
)

In [8]:
X = mean_response_extractor.fit_transform(events)

In [9]:
X.shape

(5950, 68)

In [10]:
y = events['image_id']

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
lm = LogisticRegression()

In [13]:
lm.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
lm.score(X,y)

0.07798319327731093

In [15]:
1.0 / len(y.unique())

0.008403361344537815

In [16]:
from sklearn.pipeline import Pipeline

In [ ]:
# neuronal_pipeline = Pipeline([
#     ('inference', EventInferer()),
#     ('spike', QuasiSpikeRescale()),
# ])


pipeline = Pipeline([
    ('extract', MeanResponseExtractor(traces,bins=30,range=(0,1.0),)),
    ('cluster', LogisticRegression()),
])

In [ ]:
pipeline.fit(events,y)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pipeline = Pipeline([
    ('extract', MeanResponseExtractor(traces,bins=30,range=(0,1.0),)),
    ('dimensionality_reduction',PCA(n_components=3)),
    ('cluster', LogisticRegression()),
])

In [ ]:
pipeline.fit(events,y)

In [ ]:
pipeline.score(events,y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
events_train, events_test, y_train, y_test = train_test_split(events, events['image_id'], test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

pipeline = Pipeline([
    ('extract', MeanResponseExtractor(traces,bins=15,range=(0,0.5),)),
    ('cluster', LogisticRegressionCV()),
])

In [ ]:
pipeline.fit(events_train,y_train)

In [ ]:
pipeline.score(events_test,y_test)

In [ ]:
pipeline = Pipeline([
    ('extract', MeanResponseExtractor(traces,bins=30,range=(0,1.0),)),
#     ('reduce_dim',PCA()),
    ('classify', LogisticRegressionCV()),
])

# N_FEATURES_OPTIONS = [4, 8, 16, 32]
RANGE_OPTIONS = [(0,1.0),(0,0.5),(0,0.25)]

from sklearn.neighbors import KNeighborsClassifier

param_grid = [
    {
        'extract__range': RANGE_OPTIONS,
        'classify': [LogisticRegressionCV(), KNeighborsClassifier()]
    },
]
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid)

In [ ]:
grid.fit(events_train,y_train)

In [ ]:
grid.score(events_test,y_test)